In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import random
from random import sample
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
import ast
from scipy import spatial
import csv
from tqdm.notebook import tqdm
%matplotlib inline

In [35]:
df_total = pd.read_excel('data/all_data.xlsx', sheet_name='percentChange')
stocks = pd.read_csv('data/stocks.csv')
links_total = pd.read_csv('data/links_total.csv')

In [27]:
all_sectors = {}
for sect in df_total['Sector']:
    if(sect in all_sectors):
        all_sectors[sect]+=1
    else:
        all_sectors[sect]=1
sorted_sectors = sorted(all_sectors.items(),key=lambda x:x[1],reverse=True)

### Identifying Popular Sectors
The snippet below prints all the sectors from our raw data, these include banking, software, pharmaceuticals and many more.
With these sectors now identified, we can begin exploring the relationship between them, and see how stocks within the same sectors relate to eachother

In [29]:
for sector in sorted_sectors:
    print(sector)

In [69]:
top_sectors = [
'Banks',
'Biotechnology',
'Oil, Gas & Consumable Fuels',
'Equity Real Estate Investment Trusts ...',
'Metals & Mining',
'Machinery',
'Electronic Equipment, Instruments & C...',
'Capital Markets',
'Health Care Equipment & Supplies',
'Insurance',
'Semiconductors & Semiconductor Equipment',
'Software',
'Specialty Retail',
'IT Services',
'Commercial Services & Supplies',
'Pharmaceuticals',
'Chemicals',
'Hotels, Restaurants & Leisure',
'Communications Equipment',
'Health Care Providers & Services',
'Food Products',
'Media',
'Aerospace & Defense',
'Energy Equipment & Services',
'Electrical Equipment',
'Textiles, Apparel & Luxury Goods',
'Electric Utilities',
'Trading Companies & Distributors',
'Real Estate Management & Development',
'Entertainment',
'Technology Hardware, Storage & Periph...',
'Construction & Engineering',
'Beverages',
'Airlines'
]

### Getting Stocks by sector
After we have obtained the top sectors, the snippet of code below goes through the data and finds all the stocks corresponding to our top sectors

In [73]:
stocks_by_sector = []
for sector in top_sectors:
    stocks = df_total.loc[df_total['Sector'] == sector]
    value = (sector,temp)
    stocks_by_sector.append(value)

### Calculating Average Intra-Sector Correlation
The following code blocks calculate the average weight of a link between all the stocks in a given sector. For example, in the Bank sector, the functions will go and obtain the links from one bank stock to every other bank stock, and add them to the total. Then this total will be divided by the number of links, to obtain the average weight. This is in hopes of answering our second research question

In [75]:
def calculateIntraSectorWeight(sector):
    

SyntaxError: incomplete input (341970358.py, line 2)